In [24]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
import psycopg2
from dotenv import load_dotenv
import os

In [19]:
# test connection
# Load environment variables from .env
load_dotenv()

# Connect to the database
try:
    connection = psycopg2.connect(
    user=os.getenv("USER"),
    password=os.getenv("PASSWORD"),
    host=os.getenv("HOST"),
    port=os.getenv("PORT"),
    dbname=os.getenv("DBNAME"),
)
    print("Connection successful!")
    
    # Create a cursor to execute SQL queries
    cursor = connection.cursor()
    
    # Example query
    cursor.execute("SELECT NOW();")
    result = cursor.fetchone()
    print("Current Time:", result)

    # Close the cursor and connection
    cursor.close()
    connection.close()
    print("Connection closed.")

except Exception as e:
    print(f"Failed to connect: {e}")

Connection successful!
Current Time: (datetime.datetime(2026, 1, 21, 6, 46, 47, 419454, tzinfo=datetime.timezone.utc),)
Connection closed.


In [29]:
# create configuration
engine = create_engine(
    "postgresql+psycopg2://",
    creator = lambda : psycopg2.connect(
        user=os.getenv("USER"),
        password=os.getenv("PASSWORD"),
        host=os.getenv("HOST"),
        port=os.getenv("PORT"),
        dbname=os.getenv("DBNAME")
    ),
    poolclass = NullPool
)


In [52]:
# import BMW raw data
bmw_table = pd.read_csv(...)
bmw_table.columns = bmw_table.columns.str.replace(' ', '_')
bmw_table['Date'] = pd.to_datetime(bmw_table['Date'], format = "%d/%m/%Y")
bmw_table = bmw_table.rename({'Date' : 'Purchase_Date'}, axis = 1)
bmw_table.head(5)

,Purchase_Date,Year,Model,Revenue,Quantity_Sold,Region,Country,Channel
0,2019-01-01,2019.0,BMW X2,94654.0,2.0,Africa,Nigeria,Wholesale
1,2019-01-01,2019.0,BMW M4,111259.0,1.0,Africa,Kenya,Wholesale
2,2019-01-02,2019.0,BMW 6 Series,94881.0,3.0,South America,Chile,Wholesale
3,2019-01-04,2019.0,BMW X2,35293.0,4.0,Asia,South Korea,Wholesale
4,2019-01-04,2019.0,BMW M2,76275.0,5.0,Asia,Japan,Wholesale


In [ ]:
# load BMW raw data into the DWH
bmw_table.to_sql(
    "bmw_raw_data",
    con = engine,
    schema = "data_lake",
    if_exists = "replace",
    index = False,
    method = "multi")
    

5778

In [56]:
# import Hyundai raw data
h_sales = pd.read_excel(...)
h_prices = pd.read_excel(...)
h_sales.columns = h_sales.columns.str.replace(' ', '_')
h_prices.columns = h_prices.columns.str.replace(' ', '_')
hyundai_table = pd.merge(
    h_sales.drop(columns = ['Price_($)', 'Company']),
    h_prices,
    how = 'inner',
    on = 'Model')
hyundai_table.head(5)

,Car_id,Purchase_Date,Customer_Name,Customer_Gender,Customer_Country,Agent_Region,Customer_Age,_Customer_Annual_Income($)_,Model,Engine,Transmission,Color,year_of_model,Agent_ID_,Agent_Name_,Car_Id,Company,year_of_Model,Price_($)
0,NaN,2021-02-15 00:00:00,Hossam El-Din,male,"Egypt,Giza",Giza,54 years,443768,SEDAN i10,Overhead Camshaft,Auto,Blue,2021.0,???2465??,NaN,HUN-MID-810,hyundai,2020,23100
1,NaN,2021-02-15 00:00:00,Hossam El-Din,male,"Egypt,Giza",Giza,54 years,443768,SEDAN i10,Overhead Camshaft,Auto,Blue,2021.0,???2465??,NaN,HUN-MID-820,hyundai,2021,23900
2,NaN,2021-02-15 00:00:00,Hossam El-Din,male,"Egypt,Giza",Giza,54 years,443768,SEDAN i10,Overhead Camshaft,Auto,Blue,2021.0,???2465??,NaN,HUN-MID-830,hyundai,2022,25300
3,NaN,2021-02-15 00:00:00,Hossam El-Din,male,"Egypt,Giza",Giza,54 years,443768,SEDAN i10,Overhead Camshaft,Auto,Blue,2021.0,???2465??,NaN,HUN-MID-840,hyundai,2023,27500
4,NaN,2023-08-25 00:00:00,Hossam Badawy,male,"Egypt,Alexandria",Alexandria,45 years,248041,SEDAN ELANTRA,Overhead Camshaft,Auto,White,2023.0,???2465??,NaN,HUN-MID-813,hyundai,2020,25000


In [ ]:
# load Hyundai raw data into the DWH
hyundai_table.to_sql(
    "hyundai_raw_data",
    con = engine,
    schema = "data_lake",
    if_exists = "replace",
    index = False,
    method = "multi")

8788

In [58]:
# import Lada raw data
lada_table_buyers = pd.read_csv(...)
lada_table_machines = pd.read_csv(...)

In [137]:
lada_table_buyers.head(1)

,purchase_id,id,Age,Sex,Income,Purchase_Date,Region
0,195,197678,60,M,477000.0,2021-04-01,Удмуртская Республика


In [138]:
lada_table_machines.head(1)

,purchase_id,Model,Price,Engine_Power,Transmission,Fuel_Type,Num_Additional_Options
0,1,2113,900000.0,112,MT,Petrol,4


In [60]:
# merging Lada tables
lada_table = pd.merge(
    lada_table_buyers,
    lada_table_machines,
    how = 'inner',
    on = 'purchase_id')
lada_table.head(5)

,purchase_id,id,Age,Sex,Income,Purchase_Date,Region,Model,Price,Engine_Power,Transmission,Fuel_Type,Num_Additional_Options
0,195,197678,60,M,477000.0,2021-04-01,Удмуртская Республика,Niva Travel,2050000.0,102,AT,Petrol,2
1,1900,201480,56,M,629000.0,2023-10-01,Саратовская область,Riva,1850000.0,117,AT,Petrol,2
2,5002,198108,36,M,389000.0,2021-09-01,Пермский край,2109,1800000.0,142,AT,Petrol,2
3,41,199130,43,M,240000.0,2023-01-01,Кировская область,2110,1550000.0,132,MT,Petrol,3
4,1722,197602,53,F,359000.0,2022-10-01,Пермский край,2113,2650000.0,142,AT,Petrol,1


In [ ]:
# load Lada raw data into the DWH
lada_table.to_sql(
    "lada_raw_data",
    con = engine,
    schema = "data_lake",
    if_exists = "replace",
    index = False,
    method = "multi")

5532